# Recipies based bot powered by llamaindex, Amazon Bedrock Cohere English Embedv3 , AWS Bedrock Anthropic Claude 3 Sonnet.

This notebook shows you how to use `ReAct` Agent over defined tools and RAG pipeline with Claude 3 Sonnet. Also added a reranker model (mixedbread-ai/mxbai-rerank-base-v1) that runs locally on the notebook instance.

In [1]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pydantic 2.7.1
Uninstalling pydantic-2.7.1:
  Successfully uninstalled pydantic-2.7.1
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank



In [23]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

# ------------------------------------------------------------------------
# LlamaIndex - Amazon Bedrock


llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "cohere.embed-english-v3")


Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 128


from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
from os import getenv
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)
from llama_index.core import VectorStoreIndex, StorageContext

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "https://vpc-llamaindex-ank67ovnioagxtgqsqlq6iaqwa.us-west-2.es.amazonaws.com")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "gpt-index-recipies-cohere-embed")

In [5]:
%pip install torch sentence-transformers
from llama_index.core.postprocessor import SentenceTransformerRerank
postprocessor = SentenceTransformerRerank(
    model="mixedbread-ai/mxbai-rerank-base-v1", top_n=5
)

Note: you may need to restart the kernel to use updated packages.


In [6]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
calorie_docs = SimpleDirectoryReader(input_dir="./recipies-data/calories").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
calories_index = VectorStoreIndex.from_documents(
    documents=calorie_docs, storage_context=storage_context
)

calorie_engine = calories_index.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/llama_index/core/readers/file/base.py:538: UserWarning: Specified num_workers exceed number of CPUs in the system. Setting `num_workers` down to the maximum CPU count.
  warnings.warn(


In [7]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
recipies_docs = SimpleDirectoryReader(input_dir="./recipies-data/recipies").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
recipies_index = VectorStoreIndex.from_documents(
    documents=recipies_docs, storage_context=storage_context
)

recipies_engine = recipies_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])


In [8]:
# to be added once Bedrock Multi-Modal APIs are allowed. We will an index of images of dishes and do multi-modal RAG. 

#from llama_index.core.tools import QueryEngineTool, ToolMetadata
#from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
#text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
#embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
#client = OpensearchVectorClient(
#    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
#)
# initialize vector store
#vector_store = OpensearchVectorStore(client)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
#vector_store = OpensearchVectorStore(client)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
#recipies_docs = SimpleDirectoryReader(input_dir="./recipies-data/recipies-pic").load_data(num_workers=8)

# initialize vector store
#vector_store = OpensearchVectorStore(client)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
#recipies_index = VectorStoreIndex.from_documents(
#    documents=recipies_docs, storage_context=storage_context
#)

#recipies_engine = recipies_index.as_query_engine(similarity_top_k=5, llm=llm)


In [11]:
# run query
#query_engine = index_cohere.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])
res = recipies_engine.query("How can we make pasta?")
res.response

'To make pasta, you can follow these steps:\n\nRoll out the dough as thin as possible. For different pasta shapes:\n- For fettuccine, use the fettuccine setting on a pasta machine or cut the dough into 1/4-inch wide ribbons by hand.\n- For spaghetti, use the spaghetti setting on a pasta machine or cut the dough into thin strings.\n- For ravioli, cut the rolled-out dough into 2-inch square pieces, add a dollop of filling in the center, brush the edges with egg wash, and place another pasta square on top, crimping the edges.\n- For lasagna, leave the pasta in long sheets.\n\nTo cook the pasta, bring a large pot of salted water to a boil. Add the pasta shape of your choice and cook until it rises to the surface and you can bite into it without it falling apart. The cooking time will vary depending on the pasta type.'

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=calorie_engine,
        metadata=ToolMetadata(
            name="calorie_metadata",
            description=(
                "Provides information about calories across dairy, fruits and vegetables. You can use this tool to find out calories of different ingridients in a recipe."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=recipies_engine,
        metadata=ToolMetadata(
            name="recipies",
            description=(
                "Provides information about food recipies of various dishes. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### With ReAct Agent

In [26]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill fish dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find recipes for grilled fish dishes with less than 500 calories.
Action: recipies
Action Input: {'input': 'Give me some options for grilled fish dishes that have less than 500 calories'}
Observation: Based on the provided context, here are some options for grilled fish dishes that have less than 500 calories:

1. Grilled Halibut with Lemongrass Tomato Sauce: The context mentions a recipe for grilled halibut with a lemongrass tomato sauce. Halibut is a lean, low-calorie fish, and the sauce ingredients like tomatoes, shallots, and garlic are also low in calories, making this a good option for a grilled fish dish under 500 calories.

2. Spicy Grilled Tuna with Garden Salsa: The context includes a recipe for spicy grilled tuna topped with a garden salsa. Tuna is another lean, low-calorie fish, and the salsa ingredients like tomatoes, onions, and herbs are also low in calories, making this a potentially 

In [27]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("How many calories in sliced gouda cheese?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find the calorie information for sliced gouda cheese.
Action: calorie_metadata
Action Input: {"input": "How many calories are in sliced gouda cheese?"}

Observation: Here are the calorie details for sliced gouda cheese:

Gouda cheese contains around 100-120 calories per 1 oz (28g) slice. The exact number of calories can vary slightly depending on the fat content, which ranges from around 25-30% for gouda cheese.

Some key calorie information for gouda cheese:

- 1 oz (28g) slice of regular gouda cheese: Around 100-110 calories
- 1 oz (28g) slice of low-fat gouda cheese: Around 70-80 calories
- 1 cup (112g) shredded regular gouda cheese: Around 450 calories
- 1 cup (112g) shredded low-fat gouda cheese: Around 300 calories

The calories mainly come from the fat and protein content in the cheese. Gouda is a semi-soft cheese, so it has a moderate amount of fat compared to harder aged cheeses.

Thought: I

In [28]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill chicken dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find recipes and calorie information for grilled chicken dishes under 500 calories.
Action: recipies
Action Input: {'input': 'grilled chicken recipes under 500 calories'}
Observation: Here are some grilled chicken recipes that are under 500 calories per serving:

1. Grilled Rosemary Chicken: Toss chopped rosemary and garlic with chicken, season with salt, pepper, and paprika. Grill the chicken until browned and cooked through, basting with rosemary-infused oil. Serve with lemon slices and tomato jam on the side. This recipe is likely under 500 calories per serving.

2. Garlic and Rosemary Grilled Chicken with Scallions: Season chicken with salt and pepper, then grill on top of a bed of rosemary, garlic, and scallions. Brush with oil and grill until the skin is crispy. Serve with grilled scallions on the side. This recipe should also be under 500 calories per serving.

3. Garlic and Herb Spatchcock Gr

In [29]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a morning breakfast smoothie which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is: English. To provide options for a low-calorie breakfast smoothie, I will need information on calorie counts of different ingredients and recipes for smoothies.
Action: calorie_metadata
Action Input: {'input': 'What are the calorie counts for common fruits, vegetables, and dairy products used in smoothies?'}
Observation: Based on the provided context, here are the calorie counts for some common ingredients used in smoothies:

Fruits:
- Peaches: No specific calorie information provided
- Grapefruit: No specific calorie information provided
- Orange: No specific calorie information provided
- Banana: 50 calories per 100ml
- Strawberries: No specific calorie information provided

Dairy:
- Whole milk yogurt: No specific calorie information provided

Other Ingredients:
- Agave syrup: No specific calorie information provided

The context also provides calorie information for some other non-smoothie ingredients like juices, milkshakes, and beverage

In [30]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("I want to make a cheesecake which has fruits as the topping and is not more than 2000 calories? Think Step by Step and recommend.")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find information about cheesecake recipes and calorie counts.
Action: recipies
Action Input: {'input': 'cheesecake recipe with fruit topping'}
Observation: The provided context describes several cheesecake recipes that include fruit toppings or accompaniments:

1. S'mores Cheesecake with Summer Berries: This recipe includes a berry mixture made with raspberries and blueberries that is served alongside the cheesecake.

2. Coconut Cheesecake with Passion Fruit Glaze: This recipe includes a passion fruit glaze that is poured over the top of the cheesecake, as well as a fruit compote made with mango, pineapple, star fruit, and passion fruit that is served alongside the cheesecake.

3. Key Lime Chia Cheesecake: This recipe includes a whipped cream topping that is garnished with key lime slices and chia seeds.

So in summary, the context provides several examples of cheesecake recipes that incorporate fruit toppin

In [31]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend me a cocktail drink which is based on Egg Nog? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find a cocktail recipe based on eggnog and get the calorie information.

Action: recipies
Action Input: {"input": "cocktail recipe based on eggnog"}

Observation: Here is a classic eggnog cocktail recipe:

Eggnog Cocktail
Ingredients:
- 4 oz eggnog
- 2 oz rum or brandy
- 1 oz cream or milk
- Nutmeg for garnish

Instructions:
1. Fill a cocktail shaker with ice cubes. 
2. Add the eggnog, rum/brandy, and cream/milk.
3. Shake vigorously for 10-15 seconds until well-chilled.
4. Strain into a glass.
5. Garnish with a sprinkle of freshly grated nutmeg.

This makes one serving of a rich, creamy eggnog cocktail. The alcohol can be adjusted to taste preference.

Thought: The recipe tool provided a cocktail recipe using eggnog, but did not give calorie information. I need to use the calorie metadata tool to get the calorie count.

Action: calorie_metadata
Action Input: {"input": "calories in 4 oz eggnog, 2 oz rum, 1 oz

In [32]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Recommend few eggnog based cocktails? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find recipes for eggnog based cocktails and their calorie counts.
Action: recipies
Action Input: {'input': 'eggnog based cocktails recipes'}
Observation: Based on the provided context, there are no eggnog-based cocktail recipes included. The context contains information about making eggnog ice cream, as well as a cocktail recipe called the "Stumptown Vanilla Flip" that includes ingredients like vanilla liqueur, espresso, and orange liqueur, but does not contain any eggnog. The context does not mention any eggnog-based cocktail recipes.
Thought: Since the recipies tool did not provide any eggnog cocktail recipes, I will try using the calorie_metadata tool to get calorie information for eggnog, which I can then use to estimate calories for potential eggnog cocktail recipes.
Action: calorie_metadata
Action Input: {'input': 'calories in eggnog'}
Observation: According to the provided context, the calorie content

In [33]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Recommend few eggnog cocktails? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to find recipes for eggnog cocktails and their calorie counts.
Action: recipies
Action Input: {'input': 'eggnog cocktail recipes'}
Observation: Based on the context provided, there are no eggnog cocktail recipes included. The context focuses on recipes for a Stumptown Vanilla Flip cocktail, a Rittenhouse Inn Wassail Punch, and an Eggnog Ice Cream recipe. There are no specific eggnog cocktail recipes mentioned.
Thought: The recipies tool did not provide enough information on eggnog cocktail recipes and their calorie counts. I need to search for eggnog cocktail recipes separately.
Action: recipies
Action Input: {'input': 'eggnog cocktail recipes with ingredients and instructions'}
Observation: Based on the provided context, there are no eggnog cocktail recipes included. The context information focuses on recipes for a "the-rattlesnake-cocktail" and "eggnog-ice-cream-233284", which do not contain instructions for 

In [34]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend any good indian curry dish? think step by step. also give the calorie count at the end of different ingridients")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find a good Indian curry dish recipe and calorie information.
Action: recipies
Action Input: {'input': 'Recommend a good Indian curry dish recipe'}
Observation: Based on the context provided, a good Indian curry dish recipe would be the Indian-style butter chicken curry. The recipe details the steps to prepare this flavorful and comforting dish, including:

- Marinating the chicken in a mixture of yogurt, lemon juice, chile powder, turmeric, and other spices
- Simmering the chicken in a curry sauce made with stock, sea salt, maple syrup, fenugreek leaves, cream, and garam masala
- Topping the curry with a drizzle of cream, fresh cilantro, and sliced red chiles (for those who like it spicy)
- Serving the curry with rice or naan bread

The recipe seems to result in a rich, creamy, and aromatic Indian-style butter chicken curry that would make for a delicious and satisfying comfort food dish. Based on t

In [35]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend any good indian curry dish which is low on calories and good on fiber? think step by step. also give the calorie count at the end of different ingridients")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me find a low-calorie, high-fiber Indian curry dish and get calorie counts for the ingredients.
Action: recipies
Action Input: {'input': 'Recommend a low-calorie, high-fiber Indian curry dish'}
Observation: Based on the provided context, I would recommend the "Chicken Coconut Curry in a Hurry" recipe. This dish appears to be a relatively low-calorie and high-fiber Indian curry option.

The key points that support this recommendation are:

1. The recipe uses boneless, skinless chicken breasts or thighs, which are generally lower in calories compared to other cuts of chicken.

2. It includes coconut milk, which can provide a creamy texture and flavor without the need for heavy cream or other high-calorie ingredients.

3. The recipe calls for a variety of spices and herbs, such as curry powder, ginger, and garlic, which can add flavor and nutrition without significantly increasing the calorie content.

4. 